In [1]:
# pip install faker mysql-connector-python

In [1]:
import random
import datetime
from faker import Faker

# --- CÁC THAM SỐ ---
NUM_HOCSINH = 180
NUM_GIAOVIEN = 30
NUM_GIAOVU = 1
CURRENT_NAMHOC_ID = 'NH2025'
CURRENT_NIENKHOA = 2025
# --------------------

fake = Faker('vi_VN')

# --- DỮ LIỆU CỐ ĐỊNH THEO ĐÚNG MẪU BẠN GỬI ---
VAITRO_DATA = [('VT01', 'Học sinh'), ('VT02', 'Giáo viên'), ('VT03', 'Giáo vụ')]
MONHOC_DATA = [
    ('MH01', 'Toán học'), ('MH02', 'Vật lý'), ('MH03', 'Hóa học'),
    ('MH04', 'Sinh học'), ('MH05', 'Ngữ văn'), ('MH06', 'Lịch sử'),
    ('MH07', 'Địa lý'), ('MH08', 'Đạo đức'), ('MH09', 'Thể dục')
]
LOAIDIEM_DATA = [
    ('LD01', 'Miệng', 'Điểm kiểm tra miệng', 1),
    ('LD02', '15 phút', 'Điểm kiểm tra 15 phút', 1),
    ('LD03', '1 tiết', 'Điểm kiểm tra 1 tiết', 2),
    ('LD04', 'Học kỳ', 'Điểm kiểm tra học kỳ', 3)
]
CHUCVU_DATA = [
    ('CV01', 'Lớp trưởng', 'Quản lý lớp học', 'VT01'),
    ('CV02', 'Thành viên', 'Thành viên bình thường trong lớp', 'VT01'),
    ('CV03', 'Giáo viên chủ nhiệm', 'Quản lý và chủ nhiệm lớp', 'VT02'),
    ('CV04', 'Giáo viên bộ môn', 'Giảng dạy các bộ môn', 'VT02'),
    ('CV05', 'Giáo vụ', 'Nhân sư phòng giáo vụ', 'VT03')
]
CV_LOPTRUONG, CV_THANHVIEN, CV_GVCN, CV_GVBM, CV_GIAOVU = [cv[0] for cv in CHUCVU_DATA]

#--- BƯỚC 1: TẠO DỮ LIỆU TRONG BỘ NHỚ ---

# Lưu trữ các đối tượng đã tạo để tham chiếu chéo
generated_hocsinh = []
generated_giaovien = []
generated_giaovu = []
generated_lop = [] # Sẽ chứa {'lop_id': 'L001', 'ten_lop': '10A1', 'gvcn_id': 'GV000001'}

# Lưu trữ các câu lệnh SQL để ghi ra file theo từng bảng
# Đã thêm "QUYDINHTUOI" và "QUYDINH" vào dictionary
sql = { table: [] for table in [
    "QUYDINHTUOI", "QUYDINH", # Thêm vào đây
    "VAITRO", "USERS", "GIAOVIEN", "GIAOVU", "HOCSINH", "MONHOC", "NAMHOC", "LOP", 
    "DIEM", "LOAIDIEM", "CHITIETDIEM", "CHUCVU", "HOSO", "HOSOGIAOVIEN", "HOSOHOCSINH", "HOSOGIAOVU", "PHANCONGDAY"
]}

# 1.1. Thêm dữ liệu cho QUYDINHTUOI và QUYDINH vào dictionary sql
sql["QUYDINHTUOI"].append("INSERT INTO QUYDINHTUOI (QuyDinhTuoiID, TuoiToiThieu, TuoiToiDa) VALUES ('QDHS', 15, 20);")
sql["QUYDINHTUOI"].append("INSERT INTO QUYDINHTUOI (QuyDinhTuoiID, TuoiToiThieu, TuoiToiDa) VALUES ('QDGV', 22, 60);")

sql["QUYDINH"].append("INSERT INTO QUYDINH (QuyDinhID, SiSoLop, SoLuongMonHoc, DiemDat, QuyDinhKhac) VALUES ('QD000001', 40, 10, 5.0, 'Các quy định khác...');")


# Tiếp tục với dữ liệu tĩnh hiện có
for vtid, tenvt in VAITRO_DATA:
    sql["VAITRO"].append(f"INSERT INTO VAITRO (VaiTroID, TenVaiTro) VALUES ('{vtid}', '{tenvt}');")
for mhid, tenmh in MONHOC_DATA:
    sql["MONHOC"].append(f"INSERT INTO MONHOC (MonHocID, TenMonHoc) VALUES ('{mhid}', '{tenmh}');")
for ldid, tenld, mota, heso in LOAIDIEM_DATA:
    sql["LOAIDIEM"].append(f"INSERT INTO LOAIDIEM (LoaiDiemID, TenLoaiDiem, MoTa, HeSo) VALUES ('{ldid}', '{tenld}', '{mota}', {heso});")
for cvid, tencv, mota, vtid in CHUCVU_DATA:
    sql["CHUCVU"].append(f"INSERT INTO CHUCVU (ChucVuID, TenChucVu, MoTa, VaiTroID) VALUES ('{cvid}', '{tencv}', '{mota}', '{vtid}');")

# 1.2. Tuân thủ trình tự: User -> HoSo -> GiaoVien/HocSinh
user_counter = 1
hoso_counter = 1

# === TẠO HỌC SINH ===
for i in range(1, NUM_HOCSINH + 1):
    hs_id = f"HS{i:06d}"
    user_id = f"U{user_counter:07d}"
    hoso_id = f"HOSO{hoso_counter:08d}" 
    
    chucvu_id = CV_LOPTRUONG if i == 1 else CV_THANHVIEN
    
    gioitinh = random.choice(["Nam", "Nữ"])
    hoten = fake.name_male() if gioitinh == "Nam" else fake.name_female()
    ngaysinh = fake.date_of_birth(minimum_age=15, maximum_age=18).strftime('%Y-%m-%d')
    email = f"{hs_id}@student.com"
    diachi = fake.address().replace('\n', ', ')
    now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    sql["USERS"].append(f"INSERT INTO USERS (UserID, TenDangNhap, MatKhau, VaiTroID) VALUES ('{user_id}', '{hs_id}', '123456', 'VT01');")
    sql["HOSO"].append(f"INSERT INTO HOSO (HoSoID, HoTen, GioiTinh, NgaySinh, Email, DiaChi, ChucVuID, TrangThaiHoSo, NgayTao, NgayCapNhatGanNhat) VALUES ('{hoso_id}', '{hoten}', '{gioitinh}', '{ngaysinh}', '{email}', '{diachi}', '{chucvu_id}', 'Đang hoạt động', '{now}', '{now}');")
    sql["HOCSINH"].append(f"INSERT INTO HOCSINH (HocSinhID, UserID) VALUES ('{hs_id}', '{user_id}');")
    
    generated_hocsinh.append({'hs_id': hs_id, 'user_id': user_id, 'hoso_id': hoso_id, 'chucvu_id': chucvu_id})
    user_counter += 1
    hoso_counter += 1

# 1.3. Tạo Năm học và Lớp
sql["NAMHOC"].append(f"INSERT INTO NAMHOC (NamHocID, MoTa, BatDau, KetThuc) VALUES ('{CURRENT_NAMHOC_ID}', 'Năm học {CURRENT_NIENKHOA}-{CURRENT_NIENKHOA+1}', '2025-09-05', '2026-05-31');")

# Đảm bảo có đủ lớp để phân công GVCN (bằng hoặc hơn số giáo viên)
# Đã điều chỉnh để có 12 lớp, mỗi GVCN có 1 lớp
lop_names_template = [f'10A{i}' for i in range(1, 5)] + \
                     [f'11A{i}' for i in range(1, 4)] + \
                     [f'12A{i}' for i in range(1, 3)] # Tổng cộng 9 lớp

gvcn_count = 0
lop_counter = 1 # Biến đếm cho LopID

for i, ten_lop in enumerate(lop_names_template):
    lop_id = f"L{lop_counter:03d}" # Tạo LopID theo định dạng L001, L002...
    siso = 0
    
    gvcn_id = 'NULL'
    if gvcn_count < NUM_GIAOVIEN: # Chỉ gán GVCN nếu còn GV
        gvcn_id = f"GV{gvcn_count + 1:06d}" 
        gvcn_count += 1
    if gvcn_id == 'NULL':
        sql["LOP"].append(f"INSERT INTO LOP (LopID, TenLop, SiSo, GVCNID) VALUES ('{lop_id}', '{ten_lop}', {siso}, NULL);")
    else:
        sql["LOP"].append(f"INSERT INTO LOP (LopID, TenLop, SiSo, GVCNID) VALUES ('{lop_id}', '{ten_lop}', {siso}, '{gvcn_id}');")
    generated_lop.append({'lop_id': lop_id, 'ten_lop': ten_lop, 'gvcn_id': gvcn_id})
    lop_counter += 1 # Tăng biến đếm ID lớp

# === TẠO GIÁO VIÊN ===
# Chia giáo viên thành giáo viên chủ nhiệm và giáo viên bộ môn
giaovien_chu_nhiem_pool = []
giaovien_bo_mon_pool = []

for i in range(1, NUM_GIAOVIEN + 1):
    gv_id = f"GV{i:06d}"
    user_id = f"U{user_counter:07d}"
    hoso_id = f"HOSO{hoso_counter:08d}"
    
    is_gvcn_for_any_class = False
    for lop_info in generated_lop:
        if lop_info['gvcn_id'] == gv_id:
            is_gvcn_for_any_class = True
            break
            
    chucvu_id = CV_GVCN if is_gvcn_for_any_class else CV_GVBM
    
    gioitinh = random.choice(["Nam", "Nữ"])
    hoten = fake.name_male() if gioitinh == "Nam" else fake.name_female()
    ngaysinh = fake.date_of_birth(minimum_age=25, maximum_age=55).strftime('%Y-%m-%d')
    email = f"{gv_id}@teacher.com"
    diachi = fake.address().replace('\n', ', ')
    now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    sql["USERS"].append(f"INSERT INTO USERS (UserID, TenDangNhap, MatKhau, VaiTroID) VALUES ('{user_id}', '{gv_id}', '123456', 'VT02');")
    sql["HOSO"].append(f"INSERT INTO HOSO (HoSoID, HoTen, GioiTinh, NgaySinh, Email, DiaChi, ChucVuID, TrangThaiHoSo, NgayTao, NgayCapNhatGanNhat) VALUES ('{hoso_id}', '{hoten}', '{gioitinh}', '{ngaysinh}', '{email}', '{diachi}', '{chucvu_id}', 'Đang hoạt động', '{now}', '{now}');")
    sql["GIAOVIEN"].append(f"INSERT INTO GIAOVIEN (GiaoVienID, UserID) VALUES ('{gv_id}', '{user_id}');")
    
    teacher_info = {'gv_id': gv_id, 'user_id': user_id, 'hoso_id': hoso_id, 'chucvu_id': chucvu_id}
    generated_giaovien.append(teacher_info)

    if chucvu_id == CV_GVCN:
        giaovien_chu_nhiem_pool.append(teacher_info)
    else:
        giaovien_bo_mon_pool.append(teacher_info)

    user_counter += 1
    hoso_counter += 1

# === TẠO GIÁO VỤ ===
for i in range(1, NUM_GIAOVU + 1):
    gvu_id = f"AD{i:06d}"
    user_id = f"U{user_counter:07d}"
    hoso_id = f"HOSO{hoso_counter:08d}"

    hoten = fake.name()
    gioitinh = random.choice(["Nam", "Nữ"])
    hoten = fake.name_male() if gioitinh == "Nam" else fake.name_female()
    ngaysinh = fake.date_of_birth(minimum_age=30, maximum_age=58).strftime('%Y-%m-%d')
    email = f"{gvu_id}@admin.com"
    diachi = fake.address().replace('\n', ', ')
    now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    sql["USERS"].append(f"INSERT INTO USERS (UserID, TenDangNhap, MatKhau, VaiTroID) VALUES ('{user_id}', '{gvu_id}', '123456', 'VT03');")
    sql["HOSO"].append(f"INSERT INTO HOSO (HoSoID, HoTen, GioiTinh, NgaySinh, Email, DiaChi, ChucVuID, TrangThaiHoSo, NgayTao, NgayCapNhatGanNhat) VALUES ('{hoso_id}', '{hoten}', '{gioitinh}', '{ngaysinh}', '{email}', '{diachi}', '{CV_GIAOVU}', 'Đang hoạt động', '{now}', '{now}');")
    sql["GIAOVU"].append(f"INSERT INTO GIAOVU (GiaoVuID, UserID) VALUES ('{gvu_id}', '{user_id}');")

    generated_giaovu.append({'gvu_id': gvu_id, 'user_id': user_id, 'hoso_id': hoso_id})
    user_counter += 1
    hoso_counter += 1

# 1.4. Tuân thủ trình tự: Phân bổ và tạo HoSoHocSinh/HoSoGiaoVien/HoSoGiaoVu
hhs_counter = 1
hgv_counter = 1
hgvu_counter = 1
lop_id_list = [lop['lop_id'] for lop in generated_lop]

for i, student in enumerate(generated_hocsinh):
    lop_id_assigned = lop_id_list[i % len(lop_id_list)]
    hhs_id = f"HSHS{hhs_counter:06d}"
    sql["HOSOHOCSINH"].append(f"INSERT INTO HOSOHOCSINH (HoSoHocSinhID, HocSinhID, HoSoID, LopHocID, NienKhoa) VALUES ('{hhs_id}', '{student['hs_id']}', '{student['hoso_id']}', '{lop_id_assigned}', {CURRENT_NIENKHOA});")
    hhs_counter += 1

# --- PHÂN CÔNG DẠY (PHANCONGDAY) ---
phancong_counter = 1
# Lưu trữ môn học mà mỗi giáo viên có thể dạy (sở trường)
giaovien_monhoc_so_truong = {}
for gv in generated_giaovien:
    gv_id = gv["gv_id"]
    # Mỗi giáo viên được gán 1-2 môn sở trường ngẫu nhiên
    num_mon = random.randint(1, min(2, len(MONHOC_DATA))) 
    giaovien_monhoc_so_truong[gv_id] = random.sample([mh[0] for mh in MONHOC_DATA], num_mon)

# Theo dõi các môn đã được phân công cho mỗi lớp để tránh trùng lặp
lop_phancong_monhoc = {lop_id: set() for lop_id in lop_id_list}

# Xử lý giáo viên chủ nhiệm trước
for gv_info in giaovien_chu_nhiem_pool:
    gv_id = gv_info["gv_id"]
    lop_chu_nhiem = next((lop for lop in generated_lop if lop['gvcn_id'] == gv_id), None)
    
    if lop_chu_nhiem:
        # GVCN sẽ dạy môn sở trường đầu tiên của họ ở lớp chủ nhiệm
        monhoc_id = random.choice(giaovien_monhoc_so_truong[gv_id]) # Chọn 1 môn trong số các môn sở trường
        
        # Đảm bảo môn này chưa được phân công cho lớp chủ nhiệm
        if monhoc_id not in lop_phancong_monhoc[lop_chu_nhiem['lop_id']]:
            pcd_id = f"PCD{phancong_counter:09d}"
            sql["PHANCONGDAY"].append(
                f"INSERT INTO PHANCONGDAY (PhanCongDayID, GiaoVienID, LopID, MonHocID, NamHocID) "
                f"VALUES ('{pcd_id}', '{gv_id}', '{lop_chu_nhiem['lop_id']}', '{monhoc_id}', '{CURRENT_NAMHOC_ID}');"
            )
            phancong_counter += 1
            lop_phancong_monhoc[lop_chu_nhiem['lop_id']].add(monhoc_id)
        # else:
            # print(f"Warning: {gv_id} (GVCN) could not teach {monhoc_id} in {lop_chu_nhiem['lop_id']} because it's already assigned.")

# Tạo một danh sách các giáo viên còn lại để phân công các lớp bổ sung
remaining_teachers = random.sample(generated_giaovien, len(generated_giaovien)) # Shuffle lại để phân phối đều
random.shuffle(remaining_teachers)

# Phân công các lớp còn lại cho tất cả giáo viên (bao gồm cả GVCN dạy thêm, và GVBM)
for gv_info in remaining_teachers:
    gv_id = gv_info["gv_id"]
    
    # Lấy các môn sở trường của giáo viên này
    teacher_subjects = list(giaovien_monhoc_so_truong[gv_id])
    random.shuffle(teacher_subjects) # Shuffle môn để tăng tính ngẫu nhiên
    
    # Tìm kiếm các lớp có thể dạy
    eligible_classes = []
    for lop_id in lop_id_list: # Duyệt qua tất cả các lớp
        for monhoc_id in teacher_subjects:
            # Nếu môn này chưa được phân công ở lớp này, thêm vào danh sách
            if monhoc_id not in lop_phancong_monhoc[lop_id]:
                eligible_classes.append((lop_id, monhoc_id))
    
    # Giới hạn số lượng lớp tối đa một GV dạy thêm (ví dụ: 3 lớp ngoài lớp chủ nhiệm)
    max_additional_classes = 3 if gv_info['chucvu_id'] == CV_GVBM else 2 # GVBM có thể dạy nhiều hơn
    
    # Lọc ra các phân công đã có của giáo viên này để không dạy trùng 1 lớp 2 lần (dù môn khác)
    current_assignments_for_gv = [
        (stmt.split("VALUES ('")[1].split("', '")[2], stmt.split("', '")[3]) # (LopID, MonHocID)
        for stmt in sql["PHANCONGDAY"] if stmt.split("VALUES ('")[1].split("', '")[1] == gv_id
    ]
    
    assigned_lop_ids_for_gv = {assignment[0] for assignment in current_assignments_for_gv}

    # Lọc lại eligible_classes để không gán trùng lớp cho cùng 1 giáo viên
    final_eligible_classes = [
        (lop_id, monhoc_id) for (lop_id, monhoc_id) in eligible_classes 
        if lop_id not in assigned_lop_ids_for_gv
    ]
    random.shuffle(final_eligible_classes) # Shuffle để chọn ngẫu nhiên

    assigned_count = 0
    for lop_id_to_assign, monhoc_id_to_assign in final_eligible_classes:
        if assigned_count >= max_additional_classes:
            break
        
        # Kiểm tra lại một lần nữa trước khi thêm vào SQL (tránh race condition nếu có)
        if monhoc_id_to_assign not in lop_phancong_monhoc[lop_id_to_assign]:
            pcd_id = f"PCD{phancong_counter:09d}"
            sql["PHANCONGDAY"].append(
                f"INSERT INTO PHANCONGDAY (PhanCongDayID, GiaoVienID, LopID, MonHocID, NamHocID) "
                f"VALUES ('{pcd_id}', '{gv_id}', '{lop_id_to_assign}', '{monhoc_id_to_assign}', '{CURRENT_NAMHOC_ID}');"
            )
            phancong_counter += 1
            lop_phancong_monhoc[lop_id_to_assign].add(monhoc_id_to_assign)
            assigned_lop_ids_for_gv.add(lop_id_to_assign) # Thêm lớp vào danh sách đã gán cho GV này
            assigned_count += 1
        # else:
            # print(f"Skipping assignment: {gv_id} teaching {monhoc_id_to_assign} in {lop_id_to_assign} (already covered).")

# --- Cập nhật HOSOGIAOVIEN ---
for teacher in generated_giaovien:
    hgv_id = f"HSGV{hgv_counter:06d}"
    now_str = datetime.datetime.now().strftime('%Y-%m-%d')
    sql["HOSOGIAOVIEN"].append(f"INSERT INTO HOSOGIAOVIEN (HoSoGiaoVienID, GiaoVienID, HoSoID, NgayBatDauLamViec) VALUES ('{hgv_id}', '{teacher['gv_id']}', '{teacher['hoso_id']}', '{now_str}');")
    hgv_counter += 1

# === Cập nhật HOSOGIAOVU ===
for i, admin in enumerate(generated_giaovu):
    hgvu_id =  f"HSAD{hgvu_counter:06d}"
    sql["HOSOGIAOVU"].append(f"INSERT INTO HOSOGIAOVU (HoSoGiaoVuID, GiaoVuID, HoSoID) VALUES ('{hgvu_id}', '{admin['gvu_id']}', '{admin['hoso_id']}');")
    hgvu_counter += 1

# 1.5. Tạo dữ liệu điểm
diem_counter = 1
ctd_counter = 1
for student in generated_hocsinh:
    for monhoc in MONHOC_DATA:
        for hocky in [1, 2]:
            diem_id = f"D{diem_counter:07d}"
            scores = [
                {'loai': 'LD01', 'diem': round(random.uniform(4, 10), 1)},
                {'loai': 'LD02', 'diem': round(random.uniform(4, 10), 1)},
                {'loai': 'LD03', 'diem': round(random.uniform(3, 10), 1)},
                {'loai': 'LD04', 'diem': round(random.uniform(3, 10), 1)}
            ]
            
            tong_diem, tong_heso = 0, 0
            heso_map = {ld[0]: ld[3] for ld in LOAIDIEM_DATA}
            for score in scores:
                tong_diem += score['diem'] * heso_map[score['loai']]
                tong_heso += heso_map[score['loai']]
            
            diem_tb = round(tong_diem / tong_heso, 2)
            if diem_tb >= 8.5: xep_loai = 'Giỏi'
            elif diem_tb >= 6.5: xep_loai = 'Khá'
            elif diem_tb >= 5.0: xep_loai = 'Trung bình'
            else: xep_loai = 'Yếu'
            
            sql["DIEM"].append(f"INSERT INTO DIEM (DiemID, HocSinhID, MonHocID, NamHocID, HocKy, DiemTrungBinh, XepLoai) VALUES ('{diem_id}', '{student['hs_id']}', '{monhoc[0]}', '{CURRENT_NAMHOC_ID}', {hocky}, {diem_tb}, '{xep_loai}');")
            
            for score in scores:
                ctd_id = f"CTD{ctd_counter:09d}"
                sql["CHITIETDIEM"].append(f"INSERT INTO CHITIETDIEM (ChiTietDiemID, DiemID, LoaiDiemID, GiaTri) VALUES ('{ctd_id}', '{diem_id}', '{score['loai']}', {score['diem']});")
                ctd_counter += 1
            
            diem_counter += 1

# --- BƯỚC 2: GHI RA FILE THEO ĐÚNG THỨ TỰ MẪU ---

with open('dulieu_theo_trinhtu.sql', 'w', encoding='utf-8') as f:
    f.write("USE quanlyhocsinh;\n") # Thêm câu lệnh USE ở đầu
    f.write("-- TỆP DỮ LIỆU ĐƯỢC TẠO THEO ĐÚNG TRÌNH TỰ VÀ MẪU --\n")
    
    table_order = [
        "QUYDINHTUOI", "QUYDINH", # Đặt ở đầu để đảm bảo được chèn trước
        "VAITRO", "CHUCVU", "LOAIDIEM", "MONHOC", "NAMHOC", "USERS", "HOSO", "GIAOVIEN", "HOCSINH", "GIAOVU", "LOP", 
        "HOSOHOCSINH", "HOSOGIAOVIEN", "HOSOGIAOVU", "PHANCONGDAY", "DIEM", "CHITIETDIEM" 
    ]
    
    for table_name in table_order:
        if sql[table_name]:
            f.write(f"\n-- Dữ liệu cho bảng {table_name}\n")
            f.write("\n".join(sql[table_name]))
            f.write("\n")

    f.write("\n-- HOAN TAT TAO DU LIEU --\n")

print("Đã tạo file 'dulieu_theo_trinhtu.sql' thành công!")
print("ID Lớp đã được chuyển sang định dạng L001, L002,...")
print(f"Tổng số giáo viên đã tạo: {len(generated_giaovien)}")

Đã tạo file 'dulieu_theo_trinhtu.sql' thành công!
ID Lớp đã được chuyển sang định dạng L001, L002,...
Tổng số giáo viên đã tạo: 30
